In [20]:
from pynq import Overlay
from pynq import DefaultIP
import pynq
import numpy as np
import os
    
class CNNDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)
        self.raw_outputs = pynq.allocate(shape=(3,), dtype=np.float32)
        
    bindto = ["xilinx.com:hls:cnn_action_detection:1.0"] 
    def inference(self, inp):
        self.register_map.data_in_1=inp
        self.register_map.function_select=0
        self.register_map.CTRL.AP_START=1
        while(self.register_map.CTRL.AP_DONE == 0):pass
    
    def getResult(self):
        self.register_map.function_select=1
        self.register_map.raw_output_1=self.raw_outputs.physical_address
        self.register_map.CTRL.AP_START=1
        while(self.register_map.CTRL.AP_DONE == 0):pass
        return self.register_map.result_out
    
    def resetBuffer(self):
        self.register_map.function_select=2
        self.register_map.CTRL.AP_START=1
        while(self.register_map.CTRL.AP_DONE == 0):pass

    
def copy_npy(npIn):
    inp = pynq.allocate(shape=(90,), dtype=float)
    for i in range(90):
        inp[i] = npIn[i]
    return inp

In [35]:
overlay = Overlay("bitstream/cnn.bit")

In [36]:
myip = overlay.cnn_action_detection_0

In [37]:
myip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  function_select = Register(function_select=write-only),
  data_in_1 = Register(data_in=write-only),
  data_in_2 = Register(data_in=write-only),
  result_out = Register(result_out=0),
  result_out_ctrl = Register(result_out_ap_vld=0, RESERVED=0),
  data_required = Register(data_required=0),
  data_required_ctrl = Register(data_required_ap_vld=0, RESERVED=0),
  raw_output_1 = Register(raw_output=write-only),
  raw_output_2 = Register(raw_output=write-only)
}

In [38]:
myip.register_map.CTRL.AUTO_RESTART=0

In [49]:
import numpy as np
data = np.array([
-0.93896484, -0.17504883, -0.18115234, -0.057861328, -0.01586914, -0.0031738281,
-0.9421387, -0.17749023, -0.17773438, -0.022949219, -0.017578125, 0.0087890625,
-0.9470215, -0.18334961, -0.17114258, 0.024169922, -0.026855469, 0.0026855469,
-0.9526367, -0.18432617, -0.16235352, 0.03466797, -0.0793457, -0.052490234,
-0.9567871, -0.1821289, -0.1574707, -0.0061035156, -0.15991211, -0.1081543,
-0.9577637, -0.1796875, -0.15966797, -0.034179688, -0.21850586, -0.11987305,
-0.954834, -0.17749023, -0.16552734, -0.0017089844, -0.21948242, -0.092041016,
-0.94970703, -0.17749023, -0.17236328, 0.025878906, -0.16113281, -0.053222656,
-0.9445801, -0.18139648, -0.17602539, 0.033203125, -0.118896484, -0.048828125,
-0.93896484, -0.18408203, -0.17822266, 0.026367188, -0.107177734, -0.068603516,
-0.93310547, -0.18579102, -0.17871094, 0.0029296875, -0.095214844, -0.06323242,
-0.92749023, -0.1899414, -0.17797852, -0.004638672, -0.08203125, -0.052978516,
-0.921875, -0.19238281, -0.17749023, -0.017822266, -0.07299805, -0.059814453,
-0.9177246, -0.19384766, -0.17773438, -0.049316406, -0.045166016, -0.05859375,
-0.9177246, -0.19897461, -0.17944336, -0.052490234, 0.0, -0.049072266,
-0.91918945, -0.20458984, -0.18041992, -0.012451172, -0.048828125, -0.048828125,
-0.9223633, -0.21313477, -0.17944336, 0.034179688, -0.009277344, -0.04321289,
-0.92626953, -0.22094727, -0.17700195, 0.12768555, -0.0769043, -0.061035156,
-0.92822266, -0.22241211, -0.17333984, 0.13012695, 0.041015625, -0.068115234,
-0.92871094, -0.21850586, -0.1706543, 0.08984375, -0.091308594, -0.10668945,
-0.9279785, -0.21728516, -0.16821289, 0.011230469, 0.0075683594, -0.11987305,
-0.92822266, -0.21728516, -0.16455078, -0.0036621094, -0.015625, -0.14648438,
-0.9267578, -0.21459961, -0.16064453, -0.00024414062, -0.009277344, -0.15258789,
-0.92626953, -0.21118164, -0.15600586, -0.0026855469, -0.024414062, -0.13696289,
-0.92456055, -0.20825195, -0.15454102, 0.010253906, -0.05834961, -0.12084961,
-0.923584, -0.20458984, -0.15356445, 0.012207031, -0.03491211, -0.09472656,
-0.92333984, -0.20117188, -0.15283203, 0.0061035156, -0.047851562, -0.07055664,
-0.9230957, -0.19873047, -0.15185547, -0.024169922, -0.031982422, -0.055664062,
-0.9230957, -0.1977539, -0.1508789, -0.061035156, -0.039794922, -0.044433594,
-0.9223633, -0.19873047, -0.15039062, -0.07739258, -0.037109375, -0.036621094,
-0.921875, -0.19946289, -0.15112305, -0.07055664, -0.03173828, -0.026367188,
-0.9211426, -0.20092773, -0.15258789, -0.045654297, -0.021972656, -0.0087890625,
-0.9211426, -0.203125, -0.15356445, -0.012451172, -0.0075683594, 0.013183594,
-0.9226074, -0.20385742, -0.15258789, 0.0063476562, -0.0021972656, 0.024658203,
-0.9255371, -0.20361328, -0.15356445, -0.015136719, -0.013183594, 0.024169922,
-0.9272461, -0.20288086, -0.15649414, -0.044189453, -0.02709961, 0.02758789,
-0.9272461, -0.20410156, -0.1616211, -0.032958984, -0.0087890625, 0.04345703,
-0.9267578, -0.20654297, -0.16503906, 0.017822266, 0.01171875, 0.06738281,
-0.92529297, -0.20825195, -0.16625977, 0.05029297, 0.024658203, 0.080566406,
-0.9243164, -0.20947266, -0.16552734, 0.050048828, 0.028320312, 0.07861328,
-0.9223633, -0.21044922, -0.16503906, 0.03564453, 0.020751953, 0.06689453,
-0.92041016, -0.21191406, -0.16625977, 0.030761719, 0.021728516, 0.056884766,
-0.9182129, -0.21142578, -0.16748047, 0.024658203, 0.038085938, 0.055664062,
-0.91625977, -0.21069336, -0.16674805, 0.013183594, 0.0546875, 0.060058594,
-0.9160156, -0.21166992, -0.16552734, 0.0053710938, 0.06298828, 0.06323242,
-0.91674805, -0.2133789, -0.16381836, 0.009521484, 0.064453125, 0.060546875,
-0.9194336, -0.21484375, -0.16113281, 0.012451172, 0.05517578, 0.056152344,
-0.92163086, -0.21606445, -0.15917969, -0.00390625, 0.048583984, 0.045654297,
-0.92456055, -0.21704102, -0.15722656, -0.01586914, 0.053955078, 0.03881836,
-0.92749023, -0.21948242, -0.15551758, -0.009521484, 0.057861328, 0.043945312,
-1.0366211, -0.34204102, -0.106933594, 0.18481445, -0.1616211, -0.46557617,
-1.0681152, -0.38623047, -0.09106445, 0.22460938, -0.22167969, -0.70410156,
-1.1000977, -0.4260254, -0.07006836, 0.35424805, -0.3244629, -0.9055176,
-1.130127, -0.4638672, -0.05078125, 0.5061035, -0.46142578, -1.1713867,
-1.1555176, -0.48828125, -0.034179688, 0.8093262, -0.62939453, -1.3447266,
-1.1713867, -0.49267578, -0.020019531, 0.94799805, -0.78222656, -1.447998,
-1.1789551, -0.48999023, -0.009765625, 0.83081055, -0.8967285, -1.4987793,
-1.1816406, -0.49682617, -0.0036621094, 0.65771484, -0.9484863, -1.5678711,
-1.1796875, -0.51416016, -0.0024414062, 0.607666, -0.9838867, -1.6142578,
-1.1687012, -0.5354004, -0.00390625, 0.6516113, -1.0119629, -1.6672363,
-1.1489258, -0.5505371, -0.0053710938, 0.7453613, -1.0256348, -1.7519531,
-1.1079102, -0.5566406, -0.010986328, 0.8256836, -1.1364746, -1.8068848,
-1.0666504, -0.5500488, -0.015136719, 0.8029785, -1.1091309, -1.7507324,
-1.0007324, -0.53881836, -0.022705078, 0.7314453, -1.0959473, -1.6740723,
-0.91064453, -0.50341797, -0.034423828, 0.529541, -0.9423828, -1.8139648,
-0.8183594, -0.4580078, -0.051513672, 0.23364258, -0.91748047, -1.798584,
-0.72802734, -0.41210938, -0.068847656, -0.05102539, -0.6982422, -1.541748,
-0.6328125, -0.36938477, -0.08642578, -0.54248047, -0.3876953, -1.4484863,
-0.5410156, -0.3330078, -0.106933594, -1.0812988, -0.14892578, -1.4379883,
-0.45947266, -0.3100586, -0.1381836, -1.7607422, 0.049804688, -1.2590332,
-0.38085938, -0.3244629, -0.1862793, -2.5332031, 0.23144531, -1.0759277,
-0.30932617, -0.37646484, -0.25170898, -3.0292969, 0.5541992, -1.0007324,
-0.26220703, -0.4423828, -0.33325195, -3.4221191, 1.0236816, -0.85546875,
-0.23657227, -0.5395508, -0.43554688, -3.713623, 1.0537109, -0.82958984,
-0.20922852, -0.6843262, -0.55126953, -3.5009766, 1.5866699, -0.9958496
], dtype = np.float32)

In [50]:
data.shape

(450,)

In [51]:
# sliding window with window size 15, stride 5
data_size = 75 * 6
window_size = 15
window_stride = 5
for j in range(0, len(data) - window_size*6 + 1, window_stride*6):
    data_phy = copy_npy(data[j:j + 6*window_size])
    myip.inference(data_phy.physical_address)
    print(f"result: {hex(myip.getResult())}")
    print(f"raw output: ")
    print(myip.raw_outputs)
    print(f"remaining data required: {int(myip.register_map.data_required)}")
    print("\n")
myip.resetBuffer()

result: 0xffffffff
raw output: 
[-1.316792    1.0010309   0.21894452]
remaining data required: 12


result: 0xffffffff
raw output: 
[-1.316792    1.0010309   0.21894452]
remaining data required: 11


result: 0xffffffff
raw output: 
[-1.316792    1.0010309   0.21894452]
remaining data required: 10


result: 0xffffffff
raw output: 
[-1.316792    1.0010309   0.21894452]
remaining data required: 9


result: 0xffffffff
raw output: 
[-1.316792    1.0010309   0.21894452]
remaining data required: 8


result: 0xffffffff
raw output: 
[-1.316792    1.0010309   0.21894452]
remaining data required: 7


result: 0xffffffff
raw output: 
[-1.316792    1.0010309   0.21894452]
remaining data required: 6


result: 0xffffffff
raw output: 
[-1.316792    1.0010309   0.21894452]
remaining data required: 5


result: 0xffffffff
raw output: 
[-1.316792    1.0010309   0.21894452]
remaining data required: 4


result: 0xffffffff
raw output: 
[-1.316792    1.0010309   0.21894452]
remaining data required: 3


result:

In [45]:
myip.resetBuffer()

In [44]:
int(myip.getResult())

1

In [66]:
myip.register_map.CTRL.AP_DONE

0